# Подготовка данных

## Описание датасета
Для выполнения задания предоставляется история покупок клиентов за определенный период времени, а также вспомогательные метаданные. Задача состоит в том, чтобы предсказать, какие товары купит каждый покупатель в течение 7 дней сразу после окончания обучающих данных. Клиенты, не совершившие ни одной покупки в течение этого времени, исключаются из подсчета очков.

## Содержание
- __images/__ - Папка с папками, наименования которых соответсвуют первым 3 цифрам article_id, которые расположены внутри;
- __article.csv__ — Пдробные метаданные для каждого article_id, доступного для покупки;
- __customers.csv__ — Метаданные для каждого customer_id в наборе данных;
- __sample_submission.csv__ — Образец решения для Kaggle;
- __transaction_train.csv__ — Обучающие данные.

## Библиотеки

In [1]:
import pandas as pd
import numpy as np

## Загрузка данных

In [2]:
transactions = pd.read_csv('./data/transactions_train.csv', dtype={'article_id': str})
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

articles = pd.read_csv('./data/articles.csv', dtype={'article_id': str})
customers = pd.read_csv('./data/customers.csv')

## transactions_train

### Признак `week`

Данный признак будет отвечат за то, сколько недель прошло с момента совершнения транзакции до самой последней транзакции во всем датасете

In [3]:
active_articles = transactions.groupby('article_id')['t_dat'].max().reset_index() # Дата последней покупки товара
active_articles = active_articles[active_articles['t_dat'] >= '2019']

In [4]:
transactions = transactions[transactions['article_id'].isin(active_articles['article_id'])].reset_index(drop=True)

In [5]:
transactions['week'] = (transactions['t_dat'].max() - transactions['t_dat']).dt.days // 7

In [6]:
transactions.to_csv('./data_prepared/transactions_train.csv', index=False)

##  articles

### Признак `rating`

Данный признак будет соответсовать количеству покупок конкретного товара

In [7]:
article_count = transactions['article_id'].value_counts()

In [8]:
articles = articles.merge(article_count, how='left', left_on='article_id', right_index=True)
articles = articles.rename({'article_id_x': 'article_id', 'article_id_y': 'rating'}, axis=1)

In [9]:
articles['rating'] = articles['rating'].fillna(0)

### Обработка пропусков

In [10]:
articles['detail_desc'] = articles['detail_desc'].fillna('Other')

In [11]:
articles.to_csv('./data_prepared/articles.csv', index=False)

## customers

### Признак age_group

Данный признак будет отвечать за возрастные группы. Выделим следущие периоды:
1. 0-6 - Дошкольники;
2. 7-17 - Школьники;
3. 18-44 - Молодые;
4. 45-59 – Средние;
5. 60-74 – Пожилые;
6. 75-89 – Старческие;
7. 90-100 - Долгожители.

In [12]:
# Заполним пропущенные значения средним возрастом 
customers['age'] = customers['age'].fillna(customers['age'].mean())

In [13]:
age_groups = {
    'PRESCHOOLERS': [0, 6],
    'SCHOOLERS': [7, 17],
    'YOUNG': [18, 44],
    'MEDIUM': [45, 59],
    'ELDERLY': [60, 74],
    'SENILE': [75, 89],
    'LONG_LIVE': [90, 1000]
}

In [14]:
customers['age_group'] = np.nan

for group in age_groups:
    index = customers[(customers['age'] > age_groups[group][0] - 1) & (customers['age'] < age_groups[group][1] + 1)].index
    customers.loc[index, 'age_group'] = group

### Обработка пропусков

In [15]:
customers['FN'] = customers['FN'].fillna(0)
customers['Active'] = customers['Active'].fillna(0)
customers['club_member_status'] = customers['club_member_status'].fillna('LEFT CLUB')
customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace({'None': 'NONE'}).fillna(customers['fashion_news_frequency'].mode()[0])

In [16]:
# Признак 'postal_code' скорее-всего нам не прегодится, поётому смело удалим его
customers = customers.drop('postal_code', axis=1)

In [17]:
customers.to_csv('./data_prepared/customers.csv', index=False)